# Prerequisites

* Annotations: https://drive.google.com/file/d/1IH4dL4OKG7bv57K8DreOeSAfJgkgC4sd/view
* Relevance score: http://www.trec-cds.org/qrels-treceval-abstracts.2017.txt
* Pubmed XML collection: http://trec-cds.appspot.com/2018.html#documents

In [4]:
from os import listdir
from os.path import isfile, isdir, join
from lxml import etree
import pandas as pd
import tarfile
import gzip
import time
import csv

# Decompress _.tar.gz_ Files

In [5]:
# Decompress Files
def decompress(myPath):
    fileNames = [f for f in listdir(myPath) if isfile(join(myPath, f)) and f[-7:] == ".tar.gz"]
    for file in fileNames:
        print("Extracting from: ", file)
        tar = tarfile.open(join(myPath, file), "r:gz")
        tar.extractall(join(myPath, file[:-7]))
        tar.close()
        print("Done")
        
if __name__ == "__main__":
    # Path containing the medline_xml.part[x].tar.gz files (Pubmed XML collection)
    pubMedAbstracts = "Y:\TREC\XML-Collection"

    # Decompress files
    decompress(pubMedAbstracts)

# Extract Information from Pubmed XML Files

In [6]:
# Extract Pubmed Ids from the Gold-Standard CSV file
def extractDocIDs(filePath):
    """ Extracts all ids from the gold standard """
    f = pd.read_csv(filePath)
    return set(f['trec_doc_id'])

# Get the name of the folders containing xml.gz files
def getFolderNames(myPath):
    dirNames = [d for d in listdir(myPath) if isdir(join(myPath, d))]
    return dirNames

# Get the name of the xml.gz files
def getGzFileNames(myPath):
    fileNames = [f for f in listdir(myPath) if isfile(join(myPath, f)) and f[-7:] == ".xml.gz"]
    return fileNames
        
# Extract relevant information from the papers inside the XML files that match the gold-standard
def extractFeatures(folderPath, docIDsPath, outputPath):
    st = time.time()
    
    # Get Pubmed Ids from the Gold-Standard
    ids = extractDocIDs(docIDsPath)
    print("Nr of PMIDs in the Gold-Standard:", len(ids))
    # Recover the names of each folder containing xml.gz files
    folderNames = getFolderNames(folderPath)
    
    nrExtractedXMLs = 0
    
    # Create CSV for the output
    with open(outputPath, 'w', encoding='utf-8') as extractFile:
        wr = csv.writer(extractFile, quoting=csv.QUOTE_ALL, delimiter="\t")
        wr.writerow(["trec_doc_id","title","abstract","major_mesh","minor_mesh"])
    
    # Iterate through the folders with the xml.gz files
    for folderName in folderNames:
        print("Looking into files from folder: ", folderName)
        gzFiles = getGzFileNames(join(folderPath, folderName))
        for gzFileName in gzFiles:
            print("Analyzing information from: ", gzFileName)
            gzFilePath = join(join(folderPath,folderName), gzFileName)
            parser = etree.XMLParser(encoding='utf-8', recover=True)
            pubMedArticleSet = etree.parse(gzip.open(gzFilePath, 'rt', encoding='utf-8'), parser=parser).getroot()
            for mc in pubMedArticleSet.iterfind('PubmedArticle/MedlineCitation'):
                pmid = mc.find("PMID").text
                # Verify if the PMID is in the list of IDs from the Gold-Standard
                majorMeshTerms = []
                minorMeshTerms = []
                abstractList = []
                if pmid in ids:
                    # Remove found pmid from ids set
                    ids.remove(pmid)
                    
                    print("Extracting info from the PMID: ", pmid)
                    # Get title
                    if mc.find("Article/ArticleTitle") is not None:
                        title = ''.join(mc.find("Article/ArticleTitle").itertext())
                    # Get abstract, including the structured ones
                    if mc.find("Article/Abstract") is not None:
                        for abstractPiece in mc.findall("Article/Abstract/AbstractText"):
                            abstractList.append(''.join(abstractPiece.itertext()))
                        abstract = ' '.join(abstractList)
                    # Extracting major and minor mesh descriptors
                    # Qualifiers - not taking into account major and minor attributes
                    for meshTerm in mc.findall("MeshHeadingList/MeshHeading"):
                        qualifiers = []
                        for qualifier in meshTerm.findall("QualifierName"):
                            qualifiers.append(meshTerm.find("DescriptorName").text + "/" + qualifier.text)
                        if not qualifiers:
                            fullMesh = meshTerm.find("DescriptorName").text
                            if meshTerm.find("DescriptorName").get("MajorTopicYN") == "Y":
                                majorMeshTerms.append(fullMesh)
                            else:
                                minorMeshTerms.append(fullMesh)
                        else:
                            if meshTerm.find("DescriptorName").get("MajorTopicYN") == "Y":
                                majorMeshTerms.extend(qualifiers)
                            else:
                                minorMeshTerms.extend(qualifiers)
                    majorMeshList = ";".join(majorMeshTerms)
                    minorMeshList = ";".join(minorMeshTerms)
                    
                    # Write the result to CSV
                    with open(outputPath, 'a', encoding='utf-8') as extractFile:
                        wr = csv.writer(extractFile, quoting=csv.QUOTE_ALL, delimiter="\t")
                        wr.writerow([pmid, title, abstract, majorMeshList, minorMeshList])
                
                    # Count the number of extracted papers
                    nrExtractedXMLs += 1
    
    print("Number of papers with information extracted: ", nrExtractedXMLs)
    end = time.time()
    print("Run time: ", end-st)

if __name__ == "__main__":
    # Path containing the medline_xml.partx folders - they need to be extracted first
    pubMedAbstracts = "Y:\TREC\XML-Collection"
    # Path containing the Annotated Gold-Standard File
    docIDPath = "Y:\TREC\goldDocsAnnotations.csv"
    # Output file
    outputPath = "relevantAbstractsFromXML.csv"
    
    # Extract relevant information from the XML files
    extractFeatures(pubMedAbstracts, docIDPath, outputPath)

Nr of PMIDs in the Gold-Standard: 18729
Looking into files from folder:  medline_xml.part1
Analyzing information from:  medline17n0001.xml.gz
Analyzing information from:  medline17n0002.xml.gz
Extracting info from the PMID:  57725
Extracting info from the PMID:  59126
Analyzing information from:  medline17n0003.xml.gz
Extracting info from the PMID:  89968
Analyzing information from:  medline17n0004.xml.gz
Extracting info from the PMID:  100218
Extracting info from the PMID:  113084
Analyzing information from:  medline17n0005.xml.gz
Extracting info from the PMID:  127423
Extracting info from the PMID:  140028
Analyzing information from:  medline17n0006.xml.gz
Extracting info from the PMID:  179108
Analyzing information from:  medline17n0007.xml.gz
Extracting info from the PMID:  186347
Extracting info from the PMID:  186447
Extracting info from the PMID:  203940
Extracting info from the PMID:  206742
Analyzing information from:  medline17n0008.xml.gz
Extracting info from the PMID:  2214

Extracting info from the PMID:  1493261
Analyzing information from:  medline17n0050.xml.gz
Extracting info from the PMID:  1493659
Extracting info from the PMID:  1495946
Extracting info from the PMID:  1508738
Extracting info from the PMID:  1511387
Extracting info from the PMID:  1511558
Extracting info from the PMID:  1512459
Extracting info from the PMID:  1512611
Extracting info from the PMID:  1521620
Analyzing information from:  medline17n0051.xml.gz
Extracting info from the PMID:  1524989
Extracting info from the PMID:  1527907
Extracting info from the PMID:  1532241
Extracting info from the PMID:  1539411
Extracting info from the PMID:  1550088
Analyzing information from:  medline17n0052.xml.gz
Extracting info from the PMID:  1561800
Extracting info from the PMID:  1562996
Extracting info from the PMID:  1567185
Extracting info from the PMID:  1574489
Extracting info from the PMID:  1577600
Extracting info from the PMID:  1582293
Extracting info from the PMID:  1583354
Analyzi

Extracting info from the PMID:  2214151
Extracting info from the PMID:  2214218
Extracting info from the PMID:  2214424
Extracting info from the PMID:  2214441
Extracting info from the PMID:  2224768
Extracting info from the PMID:  2228029
Extracting info from the PMID:  2236345
Extracting info from the PMID:  2243465
Analyzing information from:  medline17n0075.xml.gz
Extracting info from the PMID:  2247093
Extracting info from the PMID:  2248738
Extracting info from the PMID:  2250364
Extracting info from the PMID:  2253831
Extracting info from the PMID:  2258979
Extracting info from the PMID:  2261243
Extracting info from the PMID:  2261393
Extracting info from the PMID:  2267981
Analyzing information from:  medline17n0076.xml.gz
Extracting info from the PMID:  2275569
Extracting info from the PMID:  2276016
Extracting info from the PMID:  2278967
Extracting info from the PMID:  2284796
Extracting info from the PMID:  2288126
Extracting info from the PMID:  2291686
Extracting info fr

Extracting info from the PMID:  3089173
Extracting info from the PMID:  3099282
Analyzing information from:  medline17n0104.xml.gz
Extracting info from the PMID:  3119309
Extracting info from the PMID:  3121200
Extracting info from the PMID:  3122217
Extracting info from the PMID:  3141657
Analyzing information from:  medline17n0105.xml.gz
Extracting info from the PMID:  3166076
Extracting info from the PMID:  3166914
Extracting info from the PMID:  3167232
Extracting info from the PMID:  3167804
Extracting info from the PMID:  3167852
Extracting info from the PMID:  3168180
Analyzing information from:  medline17n0106.xml.gz
Extracting info from the PMID:  3176792
Extracting info from the PMID:  3181273
Extracting info from the PMID:  3188545
Extracting info from the PMID:  3195410
Analyzing information from:  medline17n0107.xml.gz
Extracting info from the PMID:  3229052
Analyzing information from:  medline17n0108.xml.gz
Extracting info from the PMID:  3247889
Analyzing information fro

Extracting info from the PMID:  4427743
Analyzing information from:  medline17n0148.xml.gz
Extracting info from the PMID:  4438215
Analyzing information from:  medline17n0149.xml.gz
Analyzing information from:  medline17n0150.xml.gz
Analyzing information from:  medline17n0151.xml.gz
Extracting info from the PMID:  4530870
Analyzing information from:  medline17n0152.xml.gz
Analyzing information from:  medline17n0153.xml.gz
Analyzing information from:  medline17n0154.xml.gz
Analyzing information from:  medline17n0155.xml.gz
Analyzing information from:  medline17n0156.xml.gz
Extracting info from the PMID:  4686177
Analyzing information from:  medline17n0157.xml.gz
Analyzing information from:  medline17n0158.xml.gz
Extracting info from the PMID:  4752347
Analyzing information from:  medline17n0159.xml.gz
Analyzing information from:  medline17n0160.xml.gz
Analyzing information from:  medline17n0161.xml.gz
Analyzing information from:  medline17n0162.xml.gz
Analyzing information from:  medlin

...

Extracting info from the PMID:  26993162
Extracting info from the PMID:  26993773
Extracting info from the PMID:  26993784
Extracting info from the PMID:  26994385
Extracting info from the PMID:  26995224
Extracting info from the PMID:  26995989
Extracting info from the PMID:  26997363
Extracting info from the PMID:  26997440
Extracting info from the PMID:  26997441
Extracting info from the PMID:  26997442
Extracting info from the PMID:  26998057
Extracting info from the PMID:  26999718
Extracting info from the PMID:  27000661
Extracting info from the PMID:  27000770
Extracting info from the PMID:  27000949
Extracting info from the PMID:  27001083
Extracting info from the PMID:  27001432
Extracting info from the PMID:  27002945
Extracting info from the PMID:  27004402
Extracting info from the PMID:  27004972
Extracting info from the PMID:  27005526
Analyzing information from:  medline17n0863.xml.gz
Extracting info from the PMID:  27005720
Extracting info from the PMID:  27006248
Extrac

Extracting info from the PMID:  27157302
Extracting info from the PMID:  27158147
Extracting info from the PMID:  27158185
Extracting info from the PMID:  27158758
Extracting info from the PMID:  27158780
Extracting info from the PMID:  27158978
Extracting info from the PMID:  27161538
Extracting info from the PMID:  27161690
Analyzing information from:  medline17n0868.xml.gz
Extracting info from the PMID:  27165365
Extracting info from the PMID:  27167206
Extracting info from the PMID:  27169016
Extracting info from the PMID:  27169583
Extracting info from the PMID:  27171673
Extracting info from the PMID:  27173592
Extracting info from the PMID:  27174589
Extracting info from the PMID:  27177084
Extracting info from the PMID:  27177222
Extracting info from the PMID:  27179232
Extracting info from the PMID:  27179269
Extracting info from the PMID:  27179285
Extracting info from the PMID:  27180062
Extracting info from the PMID:  27181894
Extracting info from the PMID:  27182622
Extrac

Extracting info from the PMID:  27354620
Extracting info from the PMID:  27354627
Extracting info from the PMID:  27355330
Extracting info from the PMID:  27355333
Extracting info from the PMID:  27356707
Extracting info from the PMID:  27358229
Extracting info from the PMID:  27358590
Extracting info from the PMID:  27358721
Extracting info from the PMID:  27358853
Extracting info from the PMID:  27362942
Extracting info from the PMID:  27363650
Extracting info from the PMID:  27363864
Analyzing information from:  medline17n0874.xml.gz
Extracting info from the PMID:  27364974
Extracting info from the PMID:  27366096
Extracting info from the PMID:  27370605
Extracting info from the PMID:  27371108
Extracting info from the PMID:  27371698
Extracting info from the PMID:  27372966
Extracting info from the PMID:  27373359
Extracting info from the PMID:  27374669
Extracting info from the PMID:  27374682
Extracting info from the PMID:  27375020
Extracting info from the PMID:  27384237
Extrac

Extracting info from the PMID:  27535980
Extracting info from the PMID:  27537391
Extracting info from the PMID:  27539393
Extracting info from the PMID:  27539577
Extracting info from the PMID:  27540409
Extracting info from the PMID:  27540971
Extracting info from the PMID:  27541609
Extracting info from the PMID:  27542260
Extracting info from the PMID:  27542513
Extracting info from the PMID:  27542643
Extracting info from the PMID:  27543509
Extracting info from the PMID:  27545397
Extracting info from the PMID:  27546620
Extracting info from the PMID:  27550049
Extracting info from the PMID:  27550487
Extracting info from the PMID:  27550858
Analyzing information from:  medline17n0880.xml.gz
Extracting info from the PMID:  27554588
Extracting info from the PMID:  27555788
Extracting info from the PMID:  27555987
Extracting info from the PMID:  27557830
Extracting info from the PMID:  27559432
Extracting info from the PMID:  27559463
Extracting info from the PMID:  27560501
Extrac

Extracting info from the PMID:  27696725
Extracting info from the PMID:  27698856
Extracting info from the PMID:  27699665
Extracting info from the PMID:  27702705
Extracting info from the PMID:  27702822
Extracting info from the PMID:  27702896
Extracting info from the PMID:  27703288
Extracting info from the PMID:  27707887
Extracting info from the PMID:  27708242
Extracting info from the PMID:  27713162
Extracting info from the PMID:  27713914
Analyzing information from:  medline17n0885.xml.gz
Extracting info from the PMID:  27716727
Extracting info from the PMID:  27717315
Extracting info from the PMID:  27717684
Extracting info from the PMID:  27718012
Extracting info from the PMID:  27718503
Extracting info from the PMID:  27721065
Extracting info from the PMID:  27721366
Extracting info from the PMID:  27722154
Extracting info from the PMID:  27722750
Extracting info from the PMID:  27726115
Extracting info from the PMID:  27728803
Extracting info from the PMID:  27728979
Extrac

# Read Relevant Abstracts Output

In [7]:
abstracts = pd.read_csv("relevantAbstractsFromXML.csv", sep='\t', encoding="utf-8", dtype={'trec_doc_id':object})
abstracts.shape

(17546, 5)

# Read Gold-Standard Annotation

In [8]:
annotations = pd.read_csv("Y:\TREC\goldDocsAnnotations.csv", sep=',', encoding="utf-8")
annotations.shape

(22642, 12)

# Read Relevance Score

In [10]:
relevance = pd.read_csv("Y:\TREC\qrels-treceval-abstracts.2017.txt", sep=' ', encoding="utf-8", header=None, 
                        names=["trec_topic_number", "x", "trec_doc_id", "relevance_score"])
relevance.shape

(22642, 4)

# Merge Relevance Score and Annotations

In [11]:
annotationsRelevance = annotations.merge(relevance, left_on=['trec_topic_number','trec_doc_id'], right_on=['trec_topic_number','trec_doc_id'], how='left')
annotationsRelevance.drop(["x"], axis=1, inplace=True)
annotationsRelevance.shape

(22642, 13)

# Merge Abstracts with Relevance Score and Annotations

In [12]:
processedGoldStandard = annotationsRelevance.merge(abstracts, left_on=['trec_doc_id'], right_on=['trec_doc_id'], how='left')
processedGoldStandard.shape

(22642, 17)

# Save the Result into a new _.csv_

In [16]:
date = time.strftime("%Y%m%d")
processedGoldStandard.to_csv(path_or_buf='Y:\\TREC\\'+ date + 'processedGoldStandard.csv', sep='\t')